# Creating a word2vec model from DailyDialogue dataset, which contains 10,000+ text-based dialogues between two people. 

In [105]:
import pip
install_packages = ['nltk','pandas','gensim','inflect']
for i in install_packages:
    pip.main(['install',i])
import nltk,gensim,logging,pandas as pd, re, string, unicodedata, inflect
from nltk.stem import PorterStemmer, WordNetLemmatizer, LancasterStemmer
from nltk.corpus import stopwords
l = WordNetLemmatizer()
s = PorterStemmer()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [25]:
# read the data from corpus
dial_act = open("data/dialogues_act.txt", 'r' , encoding = "utf8")
dial_text = open("data/dialogues_text.txt", 'r' , encoding = "utf8")
dial_emo = open("data/dialogues_emotion.txt", 'r' , encoding = "utf8")


merged = []

dial_act_read = dial_act.read().split(" ")
dial_act_read = dial_act_read[:-1]
dial_act_read = list(map(int, dial_act_read))

dial_emo_read = dial_emo.read().split(" ")
dial_emo_read = dial_emo_read[:-1]
dial_emo_read = list(map(int,dial_emo_read))

dial_text_read = dial_text.read().split("__eou__")
dial_text_read = dial_text_read[:-1]

person = ['A' , 'B'] * (int(len(dial_emo_read)/2))

print(len(dial_emo_read))
print(len(dial_text_read))
print(len(dial_act_read))

merged = [person ,dial_act_read, dial_emo_read, dial_text_read]
merged = list(zip(*merged))
merged = pd.DataFrame(merged)
merged.columns = ['Person' , 'Act' , 'Emotion' , 'Conversation']
print(len(merged))
dial_act.close()
dial_text.close()
dial_emo.close()

dial_act_read = []
dial_emo_read = []
#dial_text_read = []

102979
102980
102979
102978


In [21]:
merged.head()

,Person,Act,Emotion,Conversation
0,A,3,2,The kitchen stinks .
1,B,4,0,I'll throw out the garbage .
2,A,3,4,"\nSo Dick , how about getting some coffee for ..."
3,B,4,2,Coffee ? I don ’ t honestly like that kind of...
4,A,3,0,"Come on , you can at least try a little , bes..."


In [27]:
dial_text_read[:5]

['The kitchen stinks . ',
 " I'll throw out the garbage . ",
 '\nSo Dick , how about getting some coffee for tonight ? ',
 ' Coffee ? I don ’ t honestly like that kind of stuff . ',
 ' Come on , you can at least try a little , besides your cigarette . ']

# Data Pre-Processing

In [157]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    #words = stem_words(words)
    #words = lemmatize_verbs(words)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

In [158]:
a = [dial_text_read[i].replace('\n','').split(" ") for i in range(10)]
words = normalize(a[2])
print(a[0])
print(words)

['The', 'kitchen', 'stinks', '.', '']
['dick', 'getting', 'coffee', 'tonight']


In [160]:
text_clean = [normalize(dial_text_read[i].replace('\n','').split(" ")) for i in range(len(dial_text_read))]

In [162]:
text_clean[0:5]

[['kitchen', 'stinks'],
 ['ill', 'throw', 'garbage'],
 ['dick', 'getting', 'coffee', 'tonight'],
 ['coffee', 'honestly', 'like', 'kind', 'stuff'],
 ['come', 'least', 'try', 'little', 'besides', 'cigarette']]

# Training the data

In [163]:
model = gensim.models.Word2Vec(text_clean, min_count=10, size=200, window=3)

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 45802 words, keeping 5720 word types
PROGRESS: at sentence #20000, processed 100671 words, keeping 9453 word types
PROGRESS: at sentence #30000, processed 149317 words, keeping 10969 word types
PROGRESS: at sentence #40000, processed 199719 words, keeping 13024 word types
PROGRESS: at sentence #50000, processed 251412 words, keeping 15073 word types
PROGRESS: at sentence #60000, processed 306476 words, keeping 16765 word types
PROGRESS: at sentence #70000, processed 368421 words, keeping 18317 word types
PROGRESS: at sentence #80000, processed 417574 words, keeping 19617 word types
PROGRESS: at sentence #90000, processed 478587 words, keeping 21099 word types
PROGRESS: at sentence #100000, processed 542635 words, keeping 22359 word types
collected 22772 word types from a corpus of 562115 raw words and 102980 sentences
Loading a fresh vocabulary

In [172]:
model.save('model')

saving Word2Vec object under model, separately None
not storing attribute vectors_norm
not storing attribute cum_table
saved model


In [178]:
model =  gensim.models.KeyedVectors.load('model')

loading Word2VecKeyedVectors object from model
loading wv recursively from model.wv.* with mmap=None
setting ignored attribute vectors_norm to None
loading vocabulary recursively from model.vocabulary.* with mmap=None
loading trainables recursively from model.trainables.* with mmap=None
setting ignored attribute cum_table to None
loaded model


In [183]:
model.vocabulary

In [193]:
model['kitchen'].shape

C:\Users\prana\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(200,)

In [194]:
model['kitchen'][:10]

C:\Users\prana\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.0758995 ,  0.09349342, -0.07778248,  0.157159  , -0.15246977,
       -0.03411049,  0.14633189,  0.2388728 , -0.23951001,  0.07900506], dtype=float32)

In [217]:
print(model.similarity('king', 'queen'))
print(model.similarity('love', 'dog'))

0.971223799649
0.521276783487


C:\Users\prana\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
C:\Users\prana\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


#### Given the context words you can get the probability of center words

In [218]:
print(model.predict_output_word(['emergency', 'beacon', 'received']))

[('company', 0.00071256171), ('room', 0.00061875832), ('already', 0.00046515366), ('china', 0.00044574862), ('business', 0.00043729166), ('also', 0.00043230929), ('many', 0.00042863266), ('ive', 0.0004098186), ('course', 0.00040292248), ('service', 0.00040252384)]
